# Dataset Merge IDEA

1. history에 대해 drop duplicate 후 drop column      
2. watch 에 대해서도 drop duplicate 후 drop column     
3. 이후 inner join       
4. 다른 dataset merge   

문제 : 3번부터 램이 터져나감    
대안 1. 나눠서 조인 (노가다...)   
대안 2. 더 나은 노트북 환경 찾기   
대안 3. 똑똑하게 머리 쓰기 **<해결 방안 찾아냄>**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### 시청 시작 데이터에서 row 중복 체크 - 1만 6천개 가량의 중복 확인. Drop

In [2]:
'''
의심의 여지 없이 중복 존재 
아래 코드 실행 시 groupby로 상당히 많이 묶이고, 최종 column 개수가 899021로 줄었음을 알 수 있음
''' 
# history.groupby(['profile_id','log_time','ss_id']).count().sort_values('album_id',ascending=False)[:60]

'\n의심의 여지 없이 중복 존재 \n아래 코드 실행 시 groupby로 상당히 많이 묶이고, 최종 column 개수가 899021로 줄었음을 알 수 있음\n'

In [3]:
history = pd.read_csv('../input/lgground/history_data.csv')
history.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# row = 1005651 rows × 8 columns
# after drop duplicates 899021 rows × 8 columns
# 106630 개 row 중복

In [4]:
# 'continuous_play','short_trailer','log_time' -- drop
# payment 결측 0으로 대체
# history.drop(labels=['log_time','act_target_dtl','continuous_play','short_trailer'],axis=1).fillna(0,inplace=True)

### Watch에 대해서도 동일한 작업 수행 (duplicate drop, feature selection)

In [5]:
watch = pd.read_csv('../input/lgground/watch_e_data.csv')
watch.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# watch.drop(labels=['log_time','act_target_dtl'],axis=1,inplace=True)  # column 드랍. 따로 처리할 결측치 없음

In [6]:
# full 병합 코드는 폭탄 그 자체
# 돌렸다 하면 터져용 
# datamerge = pd.merge(history,watch,how='inner',on='profile_id')
# datamerge.to_csv('../input/lgground')

## 터진다...! 💣💥
*근데 여기서 문득 드는 생각*

1. 반드시 history와 watch를 inner join으로 묶어야 할까? 그러니까... serve할 비중복 유저 리스트만 history에서 가져오고, 제대로 된 neg/pos sampling이 가능한 watch 데이터에 붙여서 그것 중심으로 학습할 수도 있지 않나?     
애초에 history에서 유의미한 column은 **시청 시작 데이터에만 있는 profile_id & 그나마 payment 뿐이다**    
ㄴ *그나마 payment* : payment는 영상 시청의 장벽 같은 요소로 작동하기 때문에 (무료 영상이 유료 영상보다 많이 재생된다) sampling 혹은 추천 요소에 중요하게 고려할 만 하다.    
.       
           
이렇게 접근하면 어떤 방식이 가능해지냐면 < log_time column도 날릴 수 있게 된다 :: 그래도 train-vaild를 위해서 날리진 말자 >    
1. 두 dataset에서 필요 없는 column을 drop하고       
2. history에만 있는 profile_id 가 있는 row를 전부 추출해 watch에 append한다.
3. 추후 데이터셋 분할을 고려해, log_time 기준으로 dataframe 정렬 sort.    
3. 위 데이터셋에 meta + profile 데이터셋 merge 수행한다.    
     
> 최종적으로는 watch에 있는 중요한 값을 기본으로 가지고, serve대상인 유저목록까지 챙긴 dataset이 됨     

# LOG time 으로 duplicate 수정 !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [7]:
# data preprocessing
# history에만 있는 profile_id  Length: 8311 / watch에만 있는 profile_id  Length: 7658
m_his = history['profile_id'].drop_duplicates() # 터짐 방지로 profile_id column만 남김 
m_wat = watch['profile_id'].drop_duplicates()
id_only_in_history = pd.merge(m_his,m_wat,how='outer',indicator=True
                ).query('_merge == "left_only"').drop(columns=['_merge'])
id_only_in_history_list = id_only_in_history['profile_id'].to_list()
id_only_in_history_rows = history[history['profile_id'].isin(id_only_in_history_list)]
id_only_in_history_rows = id_only_in_history_rows.drop_duplicates(subset=['profile_id','ss_id','log_time','album_id']) # 15241 rows
id_only_in_history_rows.drop(columns=['continuous_play','short_trailer'])
dataset = watch.append(id_only_in_history_rows,sort=False).sort_values('log_time').reset_index(drop=True)

In [8]:
# dataset verification
# watch : 800632 +15241 = row 815873 나와야 정상병합
print(dataset['total_time'].isnull().sum()) #제대로 합쳐졌다면 결측이 15241개 있을 것
print(dataset['profile_id'].nunique()) #제대로 합쳐졌다면 8311명일 것 

15241
8311


In [19]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815873 entries, 0 to 815872
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   profile_id       815873 non-null  int64  
 1   ss_id            815873 non-null  int64  
 2   log_time         815873 non-null  int64  
 3   act_target_dtl   815873 non-null  object 
 4   album_id         815873 non-null  int64  
 5   watch_time       800632 non-null  float64
 6   total_time       800632 non-null  float64
 7   continuous_play  815873 non-null  object 
 8   payment          564 non-null     float64
 9   short_trailer    15241 non-null   object 
dtypes: float64(3), int64(4), object(3)
memory usage: 62.2+ MB


## Meta Data + Profile Data merge 

In [13]:
# # 메타데이터 병합 
metadata = pd.read_csv('../input/lgground/meta_data.csv')
meta_p = pd.read_csv('../input/lgground/meta_data_plus.csv')
meta = pd.merge(metadata,meta_p,how='inner',on='album_id') #### meta 정보 최종 집합체 
data_m = pd.merge(dataset,meta,how='inner',on='album_id')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
data_m.info() # 23743257 rows 가 대체 어케 나오는거임?????????? 

# metadata  42602
# meta_p 767948 entries
# meta 832356 rows

# 병합 후 data_m  2374325 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ뭔데 뭐냐구 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23743257 entries, 0 to 23743256
Data columns (total 28 columns):
 #   Column           Dtype  
---  ------           -----  
 0   profile_id       int64  
 1   ss_id            int64  
 2   log_time         int64  
 3   act_target_dtl   object 
 4   album_id         int64  
 5   watch_time       float64
 6   total_time       float64
 7   continuous_play  object 
 8   payment          float64
 9   short_trailer    object 
 10  title            object 
 11  sub_title        object 
 12  genre_large      object 
 13  genre_mid        object 
 14  genre_small      object 
 15  country          object 
 16  run_time         int64  
 17  onair_date       float64
 18  cast_1           object 
 19  cast_2           object 
 20  cast_3           object 
 21  cast_4           object 
 22  cast_5           object 
 23  cast_6           object 
 24  cast_7           object 
 25  keyword_type     object 
 26  keyword_name     object 
 27  keyword_va

## payments는 따로 history에서 row를 떼어와서 watch로 붙이던가 해야 함 (의미 있는 정보값) 

> 어제는 헷갈렸지만, 필요한 작업이다.     
이 작업을 해야 하는 이유는... 현재 payment가 추출되어 붙여진 rows들에만 붙여진 정보값이기 때문이다. watch에 처음부터 있었던 rows들에는 싹 결측으로 되어 있으므로, album_id + log_time + profile_id 기준으로 join한다면 당시의 payment 가격만 추가할 수 있다 . 


In [12]:
# pay_his = history[['profile_id','album_id','ss_id']].drop_duplicates()
# dataset = pd.merge(dataset,pay_his,how='inner',on=['profile_id','album_id','ss_id'])

## 잊지 말아야 할 것
> history의 nunique한 아이템 수 : 대략 2만건   
mata의 nunique한 아이템 수 : 대략 4만건   
== 2달간의 소비데이터에 등장조차 하지 않은 아이템이 2만개나 있다

# feature encoding


profile_data.csv 의 경우, 성별 column 수치화 작업 필요  (근데 EDA 가볍게 하고 나서 해도 늦지 않음)     
    
사실 feature encoding 작업이 모델학습 전, 전반적으로 필요